# Exercício: Sentimento em Tweets Portugueses com GRU (PyTorch)
* **Descrição:** Classificar tweets PT em positivo/negativo usando GRU bidireccional.
* **Dataset:** Portuguese Tweets for Sentiment Analysis (10 k tweets)
  * **Mais Informações:** https://www.kaggle.com/datasets/augustop/portuguese-tweets-for-sentiment-analysis?utm_source=chatgpt.com

## Passo a passo

1. Carregar tweet, sentiment; mapear posit→1, neg→0.
2. Tokenizar com spaCy em Português, cortar a 50 tokens.
3. Embedding(num_embeddings=20 000, dim=128) + Bidirectional GRU(128).
4. Otimizador Adam, BCELoss; epochs = 10, batch = 64 (GPU).
5. Medir accuracy & F1 no conjunto de teste; imprimir frases mal-classificadas.



In [ ]:
# Opção de download do dataset

# Treino (Train50.csv são 50 mil tweets)
# Pode usar também Train100.csv, Train200.csv, Train300.csv, Train400.csv, Train500.csv)
!wget https://genaiacademy.s3.eu-west-3.amazonaws.com/tweetspt/TrainingDatasets/Train50.csv

# Teste
!wget https://genaiacademy.s3.eu-west-3.amazonaws.com/tweetspt/TestDatasets/Test.csv


In [ ]:
#!pip install torch spacy==3.7.2 pt_core_news_sm
import torch, torch.nn as nn
import pandas as pd, spacy, random, numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from collections import Counter

In [ ]:
!python -m spacy download pt_core_news_sm

In [ ]:
# ---------- 1. carregar e preparar ----------
nlp = spacy.load('pt_core_news_sm')

# Carregar dados de treino
df_train = pd.read_csv('Train50.csv', sep=';')
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
df_train = df_train.head(10000)

# Supondo que os sentimentos já vêm como 0 ou 1, verificamos os dados
print(f"Colunas disponíveis: {df_train.columns.tolist()}")
print(f"Valores únicos para sentiment: {df_train.sentiment.unique()}")
df_train.info()

# Ajustar com base nos dados reais
# Como já é numérico, apenas copiamos para 'label'
df_train['label'] = df_train.sentiment

# Carregar dados de teste
df_test = pd.read_csv('Test.csv', sep=';')
df_test = df_test.sample(frac=1, random_state=42).reset_index(drop=True)
df_test = df_test.head(10000)

# Mesmo tratamento para os dados de teste
df_test['label'] = df_test.sentiment

# Identificar a coluna com os textos dos tweets (tweet ou tweet_text)
tweet_column = 'tweet_text' if 'tweet_text' in df_train.columns else 'tweet'
print(f"Usando a coluna '{tweet_column}' para os textos dos tweets")

# Exibir exemplos para verificação
print("\nExemplos de treino:")
print(df_train[[tweet_column, 'label']].head(5))
print("\nExemplos de teste:")
print(df_test[[tweet_column, 'label']].head(5))

In [ ]:
# ---------- 2. Tokenização e Vocabulário usando apenas core PyTorch ----------

# Função de tokenização usando spaCy diretamente
def tokenize(text):
    return [token.text.lower() for token in nlp(text)]

# Construir vocabulário sem usar torchtext
class SimpleVocabulary:
    def __init__(self, min_freq=2):
        self.word2idx = {'<pad>': 0, '<unk>': 1}  # Inicia com tokens especiais
        self.idx2word = {0: '<pad>', 1: '<unk>'}
        self.word_counts = Counter()
        self.min_freq = min_freq

    def build_from_texts(self, texts):
        # Tokeniza e conta todas as palavras
        for text in texts:
            tokens = tokenize(text)
            self.word_counts.update(tokens)

        # Adiciona ao vocabulário apenas palavras que atingem min_freq
        idx = len(self.word2idx)
        for word, count in self.word_counts.items():
            if count >= self.min_freq and word not in self.word2idx:
                self.word2idx[word] = idx
                self.idx2word[idx] = word
                idx += 1

        print(f"Vocabulário construído com {len(self.word2idx)} palavras únicas")
        return self

    def __len__(self):
        return len(self.word2idx)

    def text_to_indices(self, text, max_len=50):
        # Converte um texto para sequência de índices
        tokens = tokenize(text)[:max_len]  # Trunca para o tamanho máximo
        indices = [self.word2idx.get(token, 1) for token in tokens]  # 1 é o índice de <unk>
        # Padding para garantir tamanho fixo
        return indices + [0] * (max_len - len(indices))

# Cria e treina o vocabulário
max_len = 50
vocab = SimpleVocabulary(min_freq=2).build_from_texts(df_train[tweet_column])

# Função auxiliar para converter um lote de textos
def texts_to_tensor(texts, max_len=50):
    return torch.tensor([vocab.text_to_indices(text, max_len) for text in texts])

In [ ]:
# Preparar dados de treino
X_train = texts_to_tensor(df_train[tweet_column])
y_train = torch.tensor(df_train.label.values)
# Embaralhar os dados de treino
perm = torch.randperm(len(X_train))
X_train, y_train = X_train[perm], y_train[perm]

# Preparar dados de teste separados
X_test = texts_to_tensor(df_test[tweet_column])
y_test = torch.tensor(df_test.label.values)

print(f"Dados de treino: {X_train.shape[0]} exemplos")
print(f"Dados de teste: {X_test.shape[0]} exemplos")

In [ ]:
# ---------- 3. modelo ----------
class SentGRU(nn.Module):
    def __init__(self, vocab_sz):
        super().__init__()
        self.emb = nn.Embedding(vocab_sz, 128, padding_idx=0)
        self.gru = nn.GRU(128, 128, bidirectional=True, batch_first=True)
        self.fc  = nn.Linear(256, 1)
    def forward(self, x):
        emb = self.emb(x)
        _, h = self.gru(emb)
        h = torch.cat((h[0], h[1]), dim=1)
        return torch.sigmoid(self.fc(h)).squeeze(1)

In [ ]:
# Usar GPU se estiver disponível
device='cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando: {device}")

In [ ]:
model = SentGRU(len(vocab)).to(device)
opt, loss_fn = torch.optim.Adam(model.parameters(), 1e-3), nn.BCELoss()

# ---------- 4. treino ----------
def batch_iter(X, y, bs):
    idx = list(range(len(X)))
    random.shuffle(idx)
    for i in range(0, len(idx), bs):
        sel = idx[i:i+bs]
        yield X[sel].to(device), y[sel].float().to(device)

# Criar uma pequena validação para monitorar durante o treinamento
val_size = int(0.1 * len(X_train))
X_val, y_val = X_train[-val_size:], y_train[-val_size:]
X_train, y_train = X_train[:-val_size], y_train[:-val_size]

for epoch in range(10):
    model.train()
    total_loss = 0
    batches = 0

    for xb, yb in batch_iter(X_train, y_train, 64):
        opt.zero_grad()
        output = model(xb)
        loss = loss_fn(output, yb)
        loss.backward()
        opt.step()
        total_loss += loss.item()
        batches += 1

    avg_loss = total_loss / batches

    # Validação rápida
    model.eval()
    with torch.no_grad():
        val_probs = model(X_val.to(device)).cpu()
    val_preds = (val_probs > 0.5).int()
    val_acc = accuracy_score(y_val, val_preds)
    val_f1 = f1_score(y_val, val_preds)

    print(f'Época {epoch+1}: loss={avg_loss:.4f}, val_acc={val_acc:.3f}, val_f1={val_f1:.3f}')

# ---------- 5. avaliação no conjunto de teste ----------
model.eval()
with torch.no_grad():
    test_probs = model(X_test.to(device)).cpu()
test_preds = (test_probs > 0.5).int()
test_acc = accuracy_score(y_test, test_preds)
test_prec = precision_score(y_test, test_preds)
test_rec = recall_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds)

print(f'\nResultados finais no conjunto de teste:')
print(f'Accuracy: {test_acc:.4f}')
print(f'Precision: {test_prec:.4f}')
print(f'Recall: {test_rec:.4f}')
print(f'F1-score: {test_f1:.4f}')

In [ ]:
# ---------- 6. frases mal-classificadas ----------
errors = (test_preds != y_test).nonzero(as_tuple=True)[0]

# Get 5 random error indices
num_errors_to_print = min(5, len(errors))  # Ensure we don't try to print more than available
random_error_indices = random.sample(range(len(errors)), num_errors_to_print)

for i in random_error_indices:
    error_index = errors[i].item()  # Get the actual index from the errors tensor
    tweet = df_test[tweet_column].iloc[error_index]
    true_label = y_test[error_index].item()
    pred_label = test_preds[error_index].item()

    print(f"\nTweet: {tweet}")
    print(f"Verdadeiro: {'Positivo' if true_label == 1 else 'Negativo'} ({true_label})")
    print(f"Previsto: {'Positivo' if pred_label == 1 else 'Negativo'} ({pred_label})")

In [ ]:
# TODO: Mostrar frases bem classificadas